# Plot spindle current versus temperature

We want to visualize the relationship between the current and the temperature of the spindle. In order to do that, we want to take averages by given time slots. 

In [ ]:
import os
import sys
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

current_folder = os.getcwd()
print(current_folder)

## Load data

   

In [ ]:
from parse.loadCsvSignal import loadCsvSignal

folderPath = "../../data/Mecatis/Warmup_Mecatis_05_02_25/"

temperature = "lrSigSpindleTemp.csv"
current = "stSigAxCurrentS.csv"

tempPath = current_folder + "/" + folderPath + temperature
currentPath = current_folder + "/" + folderPath + current


temperatureData = loadCsvSignal(tempPath, normalize=True, debug=True)
currentData = loadCsvSignal(currentPath, normalize=True, debug=True)


## Plot the data

In [ ]:
plt.plot(
    temperatureData.loc[:, "timeSeconds"], temperatureData.loc[:, "value"], "o-", markersize=3)
plt.plot(
    currentData.loc[:, "timeSeconds"], currentData.loc[:, "value"], "o-", markersize=3)
plt.xlabel("Time (s)")
# plt.ylabel("Temperature (°C) / Current (A)")
plt.ylabel("Normalized signals")
plt.legend(["Temperature", "Current"])

plt.title("Spindle temperature and current")
plt.grid()

plt.show()

## Compute average values for slots

For the two signals, we want to define a time interval, and compute the average for each successive intervals.

In [ ]:
import pandas as pd

def computeSlotsAverage(signal: pd.DataFrame, dt: float = 10) -> pd.DataFrame:
    """
    Compute the average of the signal in each slot.

    Parameters
    ----------
    signal (pd.DataFrame): The signal to process. It must have a "timeSeconds" column.
    dt (float, optional): The duration of each slot in seconds.

    Returns
    -------
    pd.DataFrame: A DataFrame with the average value of the signal in each slot.
    """
    signal["slot"] = signal["timeSeconds"] // dt
    return signal.groupby("slot").mean()


# test the function with basic data
data = pd.DataFrame({"timeSeconds": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                     "value": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

result = computeSlotsAverage(data, dt=2)
print(result)

### Plot the data averaged by slots

In [ ]:
# apply the function to the temperature and current data
dt = 10 # seconds

temperatureSlots = computeSlotsAverage(temperatureData, dt)
currentSlots = computeSlotsAverage(currentData, dt)

# plot data versus time
plt.plot(
    temperatureSlots.loc[:, "timeSeconds"], temperatureSlots.loc[:, "value"], "o-", markersize=3)

plt.plot(
    currentSlots.loc[:, "timeSeconds"], currentSlots.loc[:, "value"], "o-", markersize=3)
plt.xlabel("Time (s)")
plt.ylabel("Temperature (°C) / Current (A)")
plt.legend(["Temperature", "Current"])
plt.title("Spindle temperature and current with slots average of " + str(dt) + " seconds")
plt.grid()
plt.show()

### Plot the current versus temperature

In [ ]:
plt.plot(
    temperatureSlots.loc[:, "value"], currentSlots.loc[:, "value"], "-o", markersize=3)
plt.xlabel("Temperature (°C)")
plt.ylabel("Current (A)")
plt.title("Spindle temperature VS current")
plt.grid()
plt.show()

## Make a function to do all these steps

We want to plot the current VS temperature plot for multiple samples.

In [ ]:
def getCurrentAndTemp(samplePath: str, timeSlot: float = 10, debug: bool = False) -> pd.DataFrame:
    """
    Get the current and temperature data from a sample path.

    Parameters
    ----------
    samplePath (str): The path to the sample.
    timeSlot (float): The duration of each slot on which to average in seconds.
        Defaults to 10 seconds.
    debug (bool, optional): Whether to print debug information. Defaults to True.

    Returns
    -------
    pd.DataFrame: The current and temperature data.
    """
    # load the data
    temperature = loadCsvSignal(samplePath + "lrSigSpindleTemp.csv", debug=debug)
    current = loadCsvSignal(samplePath + "stSigAxCurrentS.csv", debug=debug)

    temperatureSlots = computeSlotsAverage(temperature, timeSlot)
    currentSlots = computeSlotsAverage(current, timeSlot)

    # merge the data
    data = pd.DataFrame()
    data["timeSeconds"] = temperatureSlots["timeSeconds"]
    data["temperature"] = temperatureSlots["value"]
    data["current"] = currentSlots["value"]

    return data

## Test `getCurrentAndTemp` on various samples

In [ ]:
machinePath = "../../data/Mecatis/"

samples = ["Warmup_Mecatis_05_02_25/", 
           "Warmup_Mecatis_17_02_25/",
           "Warmup_Mecatis_10_03_25/"] 

data = []
for sample in samples:
    data.append(getCurrentAndTemp(machinePath + sample))


def showTempCalibrationPlots(data: list, sampleNames: list) -> None:
    """
    Show the temperature and current plots for the given data.

    Parameters
    ----------
    data (list): The data to plot.
    """
    # plot temperature versus time
    plt.figure()
    for i in range(len(data)):
        plt.plot(
            data[i]["timeSeconds"], data[i]["temperature"], "o-", markersize=3, label=sampleNames[i])
    plt.xlabel("Time (s)")
    plt.ylabel("Temperature (°C)")
    plt.title("Spindle temperature")
    plt.legend()
    plt.grid()
    plt.show()

    # plot current versus time
    plt.figure()
    for i in range(len(data)):
        plt.plot(
            data[i]["timeSeconds"], data[i]["current"], "o-", markersize=3, label=sampleNames[i])
    plt.xlabel("Time (s)")
    plt.ylabel("Current (A)")
    plt.title("Spindle current")
    plt.legend()
    plt.grid()
    plt.show()

    # plot current versus temperature
    plt.figure()
    for i in range(len(data)):
        plt.plot(
            data[i]["temperature"], data[i]["current"], "o-", markersize=3, label=sampleNames[i])
    plt.xlabel("Temperature (°C)")
    plt.ylabel("Current (A)")
    plt.title("Spindle temperature VS current")
    plt.legend()
    plt.grid()
    plt.show()
        
showTempCalibrationPlots(data, samples)

## Finding optimal temperature offset 

We want to minimize the distance between a temperature signal and a reference using just an offset.

In [ ]:
import numpy as np
from scipy.optimize import minimize

def fun(x: float, data: pd.Series, reference: pd.Series) -> float:
    error = ((data - x - reference)**2).sum()/len(data) # mean square error (MSE)
    return error

def findTemperatureOffset(temperature: pd.Series, reference: pd.Series) -> float:
    """
    Find the temperature offset that minimizes the error between the temperature and reference.

    Parameters
    ----------
    temperature (pd.Series): The temperature data.
    reference (pd.Series): The reference data.

    Returns
    -------
    float: The temperature offset.
    """
    x0 = 0.0 # start with an offset of 0°C
    result = minimize(fun, x0, args=(temperature, reference))
    return result.x[0], np.sqrt(result.fun) # return the offset and the rmse

Test the function on some data:

In [ ]:
offset1, error1 = findTemperatureOffset(data[1]["temperature"], data[0]["temperature"])

print("sample 1: ", samples[1])
print("Offset: ", offset1.round(2), "°C")
print("RMSE: ", error1.round(2), "°C")

offset2, error2 = findTemperatureOffset(data[2]["temperature"], data[0]["temperature"])
print("sample 2: ", samples[2])
print("Offset: ", offset2.round(2), "°C")
print("RMSE: ", error2.round(2), "°C")


Plot the corrected data

In [ ]:
data[1]["temperature"] = data[1]["temperature"] - offset1
samples[1] = samples[1] + " (offset: " + str(offset1.round(2)) + "°C)"

data[2]["temperature"] = data[2]["temperature"] - offset2
samples[2] = samples[2] + " (offset: " + str(offset2.round(2)) + "°C)"

showTempCalibrationPlots(data, samples)